In [1]:
import asyncio
import json
import logging
import zipfile
from datetime import date
from typing import List
import pandas as pd

import aiohttp
from minio import Minio
from pathlib import Path
import shutil
import re

In [2]:
minio_connection = ""

In [3]:
# Configuração básica de logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [4]:
# carregar para funcionar
try:
    minio_conn = json.loads(minio_connection)
except json.JSONDecodeError:
    with open("../variables/minio_connection.json", "r") as minio_connection_file:
        minio_conn = json.loads(minio_connection_file.read())

In [5]:
s3_client = None

try:
    endpoint_raw = minio_conn["endpoint"]
    access_key = minio_conn["access_key"]
    secret_key = minio_conn["key"]

    endpoint_sem_http = endpoint_raw.replace("http://", "").replace("https://", "")
    is_secure = endpoint_raw.startswith("https")

    s3_client = Minio(
        endpoint=endpoint_sem_http,
        access_key=access_key,
        secret_key=secret_key,
        secure=is_secure
    )

    logging.info("Cliente MinIO criado com sucesso.")

except KeyError as e:
    logging.error(f"Erro de configuração: chave ausente - {e}")
except Exception as e:
    logging.error(f"Erro ao inicializar o cliente MinIO: {e}")

2025-08-18 11:40:52,970 - INFO - Cliente MinIO criado com sucesso.


In [6]:
url = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/"
bucket = "landing"
schema = "rfb"
table = "cnpj_qualificacoes"

archives = [
    "Qualificacoes.zip",
]

current_year = date.today().year
current_month = date.today().month

years_to_download = 5
start_year = current_year - years_to_download + 1


In [7]:
# --- Função para listar diretórios existentes ---
async def listar_diretorios_existentes():
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            html = await resp.text()
            return re.findall(r"(\d{4}-\d{2})/", html)  # pega só YYYY-MM/

In [8]:
async def fetch(session: aiohttp.ClientSession, url: str, sem: asyncio.Semaphore, retries=3):
    async with sem:
        for tentativa in range(1, retries+1):
            try:
                timeout = aiohttp.ClientTimeout(total=600)
                async with session.get(url, timeout=timeout) as response:

                    status = response.status
                    if status != 200:
                        logger.error(f"Url: {url}, Status: {status}, Message: Status inesperado")
                        return None

                    content_type = response.headers.get("Content-Type")
                    if content_type != "application/zip":
                        logger.error(f"Url: {url}, Status: {status}, Message: Tipo inesperado: {content_type}")
                        return None

                    content_length = response.headers.get("Content-Length")
                    if content_length is None or int(content_length) == 0:
                        logger.error(f"Url: {url}, Status: {status}, Message: Tamanho indefinido ou inesperado")
                        return None

                    filename = url.split("/")[-1]
                    path_file = f"download/{filename}"
                    with open(path_file, "wb") as zip_file:
                        async for chunk in response.content.iter_chunked(1024*1024):
                            zip_file.write(chunk)
                    logger.info(f"Baixado: {filename}")

                    # Extrai o zip
                    try:
                        with zipfile.ZipFile(path_file, "r") as zf:
                            zf.extractall("download")
                        logger.info(f"Extraído: {filename}")
                    except zipfile.BadZipFile as bz:
                        logger.error(f"Url: {url}, Error: {bz}, Message: Erro ao tentar descompactar o arquivo: {filename}")
                    return  # sucesso

            except Exception as e:
                logger.warning(f"Tentativa {tentativa}/{retries} falhou para {url}: {e}")
                await asyncio.sleep(5)

In [9]:
# --- Função para download com limite de concorrência ---
async def downloader(urls):
    sem = asyncio.Semaphore()  # Limite de concorrência
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url, sem) for url in urls]
        await asyncio.gather(*tasks)

In [ ]:
# --- Função principal ---
async def main():
    dirs_existentes = await listar_diretorios_existentes()

    urls_para_baixar = []
    for year in range(start_year, current_year + 1):
        last_month = current_month if year == current_year else 12
        for month in range(1, last_month + 1):
            dir_name = f"{year}-{str(month).zfill(2)}"
            if dir_name in dirs_existentes:  # só pega meses que existem
                for archive in archives:
                    urls_para_baixar.append(f"{url}{dir_name}/{archive}")

    logger.info(f"{len(urls_para_baixar)} arquivos para baixar")
    await downloader(urls_para_baixar)

    # --- Após baixar, filtra apenas arquivos de Qualificações ---
    pasta_base = Path("download")
    padrao_arquivo = "Qualificacoes"
    arquivos_qualificacoes = [arq for arq in pasta_base.rglob("*") if padrao_arquivo.lower() in arq.name.lower()]

    logger.info(f"Arquivos de Qualificacoes encontrados: {len(arquivos_qualificacoes)}")
    dfs = []
    for arquivo in arquivos_qualificacoes:
        try:
            df = pd.read_csv(arquivo, sep=";", encoding="latin1", low_memory=False)
            dfs.append(df)
        except Exception as e:
            logger.error(f"Erro ao ler {arquivo}: {e}")

    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)
        logger.info(f"Total de linhas consolidadas: {len(df_final)}")

if __name__ == "__main__":
    Path("download").mkdir(exist_ok=True)
    await main()

2025-08-18 11:40:53,388 - INFO - 28 arquivos para baixar
2025-08-18 11:40:53,747 - INFO - Baixado: Qualificacoes.zip
2025-08-18 11:40:53,750 - INFO - Extraído: Qualificacoes.zip
2025-08-18 11:40:53,834 - INFO - Baixado: Qualificacoes.zip
2025-08-18 11:40:53,836 - INFO - Extraído: Qualificacoes.zip
2025-08-18 11:40:53,921 - INFO - Baixado: Qualificacoes.zip
2025-08-18 11:40:53,923 - INFO - Extraído: Qualificacoes.zip
2025-08-18 11:40:54,009 - INFO - Baixado: Qualificacoes.zip
2025-08-18 11:40:54,011 - INFO - Extraído: Qualificacoes.zip
2025-08-18 11:40:54,096 - INFO - Baixado: Qualificacoes.zip
2025-08-18 11:40:54,098 - INFO - Extraído: Qualificacoes.zip
2025-08-18 11:40:54,183 - INFO - Baixado: Qualificacoes.zip
2025-08-18 11:40:54,184 - INFO - Extraído: Qualificacoes.zip
2025-08-18 11:40:54,269 - INFO - Baixado: Qualificacoes.zip
2025-08-18 11:40:54,270 - INFO - Extraído: Qualificacoes.zip
2025-08-18 11:40:54,355 - INFO - Baixado: Qualificacoes.zip
2025-08-18 11:40:54,356 - INFO - Ext

In [11]:
pasta_base = Path("download")
padrao_arquivo = "QUALSCSV"

In [12]:
# Lista apenas CSVs
arquivos_qualificacoes = [arq for arq in pasta_base.rglob("*") if padrao_arquivo in arq.name.upper()]

logger.info(f"Arquivos CSV de qualificacoes encontrados: {len(arquivos_qualificacoes)}")

2025-08-18 11:40:56,114 - INFO - Arquivos CSV de qualificacoes encontrados: 28


In [13]:
# Upload para o MinIO
for arquivo in arquivos_qualificacoes:
    caminho_relativo = arquivo.relative_to(pasta_base)
    destino = f"rfb/cnpj_qualificacoes/{caminho_relativo.as_posix()}"
    s3_client.fput_object(
        bucket_name="landing",
        object_name=destino,
        file_path=str(arquivo)
    )

In [14]:
# Limpeza da pasta download após upload
downloads_path = Path("download")
try:
    if downloads_path.exists():
        shutil.rmtree(downloads_path)
        logger.info(f"Pasta '{downloads_path}' removida com sucesso após upload.")
    else:
        logger.warning(f"Pasta '{downloads_path}' não encontrada para remoção.")
except Exception as e:
    logger.error(f"Erro ao tentar remover '{downloads_path}': {e}")

2025-08-18 11:40:56,443 - INFO - Pasta 'download' removida com sucesso após upload.
